In [20]:
import pandas as pd


In [34]:
df = pd.read_csv("../runs/results.csv")
df = df.dropna()
df = df.drop(columns=["run_dir", "test_ckpt_path", "val_ckpt_path"])
df.head(3)


,run,embed_dim,img_enc,img_trainable,txt_enc,txt_pooling,txt_trainable,txt_unfreeze_last_n,proj_type,proj_hidden_dim,...,val_ckpt_epoch,val_pairs,test_i2t_R@1,test_i2t_R@5,test_i2t_R@10,test_t2i_R@1,test_t2i_R@5,test_t2i_R@10,test_ckpt_epoch,test_pairs
4,sweep_20260217_145801__001__model_text_encoder...,512,resnet50,False,distilbert-base-uncased,mean,False,0,linear,1024,...,10.0,5000.0,0.190,0.437,0.586,0.1476,0.1476,0.2310,10.0,5000.0
6,sweep_20260217_151856__001__model_text_encoder...,512,resnet50,False,distilbert-base-uncased,mean,False,0,linear,1024,...,10.0,5000.0,0.187,0.437,0.586,0.1472,0.1472,0.2314,10.0,5000.0
7,sweep_20260217_151856__002__model_text_encoder...,512,resnet50,False,distilbert-base-uncased,mean,False,0,linear,1024,...,10.0,5000.0,0.189,0.439,0.586,0.1474,0.1474,0.2318,10.0,5000.0


In [36]:
nunique = df.nunique(dropna=False)

const_cols = nunique[nunique <= 1].index.tolist()
const_cols


['embed_dim',
 'img_enc',
 'img_trainable',
 'txt_enc',
 'txt_pooling',
 'txt_trainable',
 'proj_hidden_dim',
 'proj_dropout',
 'temp_learnable',
 'temp_init',
 'dataset',
 'batch_size',
 'image_size',
 'max_length',
 'epochs',
 'lr_image',
 'weight_decay',
 'amp',
 'val_ckpt_epoch',
 'val_pairs',
 'test_ckpt_epoch',
 'test_pairs']

In [37]:
df = df.drop(columns=const_cols)

In [38]:
df.head(10)

,run,txt_unfreeze_last_n,proj_type,lr,lr_text,val_i2t_R@1,val_i2t_R@5,val_i2t_R@10,val_t2i_R@1,val_t2i_R@5,val_t2i_R@10,test_i2t_R@1,test_i2t_R@5,test_i2t_R@10,test_t2i_R@1,test_t2i_R@5,test_t2i_R@10
4,sweep_20260217_145801__001__model_text_encoder...,0,linear,0.0010,0.000010,0.203,0.471,0.613,0.1546,0.1546,0.2396,0.190,0.437,0.586,0.1476,0.1476,0.2310
6,sweep_20260217_151856__001__model_text_encoder...,0,linear,0.0010,0.000010,0.203,0.471,0.611,0.1548,0.1548,0.2402,0.187,0.437,0.586,0.1472,0.1472,0.2314
7,sweep_20260217_151856__002__model_text_encoder...,0,linear,0.0010,0.000050,0.204,0.473,0.613,0.1546,0.1546,0.2400,0.189,0.439,0.586,0.1474,0.1474,0.2318
8,sweep_20260217_151856__003__model_text_encoder...,2,linear,0.0010,0.000010,0.240,0.516,0.631,0.1724,0.1724,0.2688,0.214,0.482,0.615,0.1790,0.1790,0.2786
9,sweep_20260217_151856__004__model_text_encoder...,2,linear,0.0010,0.000050,0.228,0.518,0.644,0.1822,0.1822,0.2784,0.233,0.499,0.610,0.1838,0.1838,0.2904
10,sweep_20260217_151856__005__model_text_encoder...,4,linear,0.0010,0.000010,0.100,0.278,0.409,0.0758,0.0758,0.1306,0.081,0.279,0.414,0.0722,0.0722,0.1326
11,sweep_20260217_151856__006__model_text_encoder...,4,linear,0.0010,0.000050,0.090,0.280,0.405,0.0818,0.0818,0.1378,0.092,0.266,0.379,0.0774,0.0774,0.1402
12,sweep_20260218_094115__001__model_text_encoder...,2,linear,0.0001,0.000010,0.347,0.646,0.755,0.2588,0.2588,0.3794,0.337,0.632,0.728,0.2594,0.2594,0.3684
13,sweep_20260218_094115__002__model_text_encoder...,2,mlp,0.0001,0.000010,0.316,0.598,0.723,0.2502,0.2502,0.3680,0.290,0.582,0.725,0.2362,0.2362,0.3534
14,sweep_20260218_094115__003__model_text_encoder...,2,linear,0.0001,0.000005,0.344,0.646,0.757,0.2588,0.2588,0.3792,0.337,0.632,0.729,0.2592,0.2592,0.3680


In [39]:
w1 = 0.5
w5 = 0.3
w10 = 0.2

df["val_i2t_weighted"] = (
    w1 * df["val_i2t_R@1"] +
    w5 * df["val_i2t_R@5"] +
    w10 * df["val_i2t_R@10"]
)

df["val_t2i_weighted"] = (
    w1 * df["val_t2i_R@1"] +
    w5 * df["val_t2i_R@5"] +
    w10 * df["val_t2i_R@10"]
)

df["val_weighted_total"] = 0.5 * (
    df["val_i2t_weighted"] +
    df["val_t2i_weighted"]
)

df["test_i2t_weighted"] = (
    w1 * df["test_i2t_R@1"] +
    w5 * df["test_i2t_R@5"] +
    w10 * df["test_i2t_R@10"]
)

df["test_t2i_weighted"] = (
    w1 * df["test_t2i_R@1"] +
    w5 * df["test_t2i_R@5"] +
    w10 * df["test_t2i_R@10"]
)

df["test_weighted_total"] = 0.5 * (
    df["test_i2t_weighted"] +
    df["test_t2i_weighted"]
)




# Best

In [42]:
best_val = df.sort_values("val_weighted_total", ascending=False)
best_val.head(10)


,run,txt_unfreeze_last_n,proj_type,lr,lr_text,val_i2t_R@1,val_i2t_R@5,val_i2t_R@10,val_t2i_R@1,val_t2i_R@5,...,test_i2t_R@10,test_t2i_R@1,test_t2i_R@5,test_t2i_R@10,val_i2t_weighted,val_t2i_weighted,val_weighted_total,test_i2t_weighted,test_t2i_weighted,test_weighted_total
24,sweep_20260218_094115__013__model_text_encoder...,6,linear,0.0001,0.000010,0.359,0.652,0.756,0.2604,0.2604,...,0.749,0.2582,0.2582,0.3744,0.5263,0.28492,0.40561,0.5076,0.28144,0.39452
26,sweep_20260218_094115__015__model_text_encoder...,6,linear,0.0001,0.000005,0.359,0.653,0.755,0.2604,0.2604,...,0.752,0.2596,0.2596,0.3736,0.5264,0.28480,0.40560,0.5084,0.28240,0.39540
28,sweep_20260218_094115__017__model_text_encoder...,6,linear,0.0001,0.000001,0.355,0.657,0.756,0.2604,0.2604,...,0.752,0.2578,0.2578,0.3740,0.5258,0.28452,0.40516,0.5110,0.28104,0.39602
12,sweep_20260218_094115__001__model_text_encoder...,2,linear,0.0001,0.000010,0.347,0.646,0.755,0.2588,0.2588,...,0.728,0.2594,0.2594,0.3684,0.5183,0.28292,0.40061,0.5037,0.28120,0.39245
22,sweep_20260218_094115__011__model_text_encoder...,4,linear,0.0001,0.000001,0.351,0.638,0.756,0.2592,0.2592,...,0.742,0.2672,0.2672,0.3806,0.5181,0.28248,0.40029,0.5156,0.28988,0.40274
14,sweep_20260218_094115__003__model_text_encoder...,2,linear,0.0001,0.000005,0.344,0.646,0.757,0.2588,0.2588,...,0.729,0.2592,0.2592,0.3680,0.5172,0.28288,0.40004,0.5039,0.28096,0.39243
20,sweep_20260218_094115__009__model_text_encoder...,4,linear,0.0001,0.000005,0.350,0.638,0.756,0.2588,0.2588,...,0.742,0.2674,0.2674,0.3806,0.5176,0.28224,0.39992,0.5161,0.29004,0.40307
16,sweep_20260218_094115__005__model_text_encoder...,2,linear,0.0001,0.000001,0.345,0.645,0.755,0.2586,0.2586,...,0.728,0.2586,0.2586,0.3678,0.5170,0.28264,0.39982,0.5045,0.28044,0.39247
21,sweep_20260218_094115__010__model_text_encoder...,4,mlp,0.0001,0.000005,0.345,0.613,0.737,0.2604,0.2604,...,0.718,0.2424,0.2424,0.3590,0.5038,0.28456,0.39418,0.4719,0.26572,0.36881
19,sweep_20260218_094115__008__model_text_encoder...,4,mlp,0.0001,0.000010,0.341,0.606,0.738,0.2608,0.2608,...,0.717,0.2416,0.2416,0.3588,0.4999,0.28516,0.39253,0.4668,0.26504,0.36592


In [31]:
best_test = df.sort_values("test_weighted_total", ascending=False)
best_test.head(10)


,txt_unfreeze_last_n,proj_type,lr,lr_text,val_i2t_R@1,val_i2t_R@5,val_i2t_R@10,val_t2i_R@1,val_t2i_R@5,val_t2i_R@10,...,test_i2t_R@10,test_t2i_R@1,test_t2i_R@5,test_t2i_R@10,val_i2t_weighted,val_t2i_weighted,test_i2t_weighted,test_t2i_weighted,test_weighted_total,val_weighted_total
20,4,linear,0.0001,0.000005,0.350,0.638,0.756,0.2588,0.2588,0.3760,...,0.742,0.2674,0.2674,0.3806,0.5176,0.28224,0.5161,0.29004,0.40307,0.39992
22,4,linear,0.0001,0.000001,0.351,0.638,0.756,0.2592,0.2592,0.3756,...,0.742,0.2672,0.2672,0.3806,0.5181,0.28248,0.5156,0.28988,0.40274,0.40029
28,6,linear,0.0001,0.000001,0.355,0.657,0.756,0.2604,0.2604,0.3810,...,0.752,0.2578,0.2578,0.3740,0.5258,0.28452,0.5110,0.28104,0.39602,0.40516
26,6,linear,0.0001,0.000005,0.359,0.653,0.755,0.2604,0.2604,0.3824,...,0.752,0.2596,0.2596,0.3736,0.5264,0.28480,0.5084,0.28240,0.39540,0.40560
24,6,linear,0.0001,0.000010,0.359,0.652,0.756,0.2604,0.2604,0.3830,...,0.749,0.2582,0.2582,0.3744,0.5263,0.28492,0.5076,0.28144,0.39452,0.40561
16,2,linear,0.0001,0.000001,0.345,0.645,0.755,0.2586,0.2586,0.3788,...,0.728,0.2586,0.2586,0.3678,0.5170,0.28264,0.5045,0.28044,0.39247,0.39982
12,2,linear,0.0001,0.000010,0.347,0.646,0.755,0.2588,0.2588,0.3794,...,0.728,0.2594,0.2594,0.3684,0.5183,0.28292,0.5037,0.28120,0.39245,0.40061
14,2,linear,0.0001,0.000005,0.344,0.646,0.757,0.2588,0.2588,0.3792,...,0.729,0.2592,0.2592,0.3680,0.5172,0.28288,0.5039,0.28096,0.39243,0.40004
25,6,mlp,0.0001,0.000010,0.325,0.626,0.728,0.2486,0.2486,0.3726,...,0.712,0.2502,0.2502,0.3604,0.4959,0.27340,0.4677,0.27224,0.36997,0.38465
21,4,mlp,0.0001,0.000005,0.345,0.613,0.737,0.2604,0.2604,0.3812,...,0.718,0.2424,0.2424,0.3590,0.5038,0.28456,0.4719,0.26572,0.36881,0.39418


In [ ]:
df.groupby("txt_unfreeze_last_n")["val_t2i_R@1"].mean()

txt_unfreeze_last_n
0    0.154667
2    0.235200
4    0.208029
6    0.254533
Name: val_t2i_R@1, dtype: float64

In [ ]:
df["asymmetry_ratio"] = df["val_t2i_R@1"] / df["val_i2t_R@1"]
df.sort_values("asymmetry_ratio")].head(10)

,txt_unfreeze_last_n,proj_type,lr,lr_text,val_i2t_R@1,val_i2t_R@5,val_i2t_R@10,val_t2i_R@1,val_t2i_R@5,val_t2i_R@10,...,test_t2i_R@1,test_t2i_R@5,test_t2i_R@10,val_i2t_weighted,val_t2i_weighted,test_i2t_weighted,test_t2i_weighted,test_weighted_total,val_weighted_total,asymmetry_ratio
8,2,linear,0.0010,0.000010,0.240,0.516,0.631,0.1724,0.1724,0.2688,...,0.1790,0.1790,0.2786,0.4010,0.19168,0.3746,0.19892,0.28676,0.29634,0.718333
24,6,linear,0.0001,0.000010,0.359,0.652,0.756,0.2604,0.2604,0.3830,...,0.2582,0.2582,0.3744,0.5263,0.28492,0.5076,0.28144,0.39452,0.40561,0.725348
26,6,linear,0.0001,0.000005,0.359,0.653,0.755,0.2604,0.2604,0.3824,...,0.2596,0.2596,0.3736,0.5264,0.28480,0.5084,0.28240,0.39540,0.40560,0.725348
28,6,linear,0.0001,0.000001,0.355,0.657,0.756,0.2604,0.2604,0.3810,...,0.2578,0.2578,0.3740,0.5258,0.28452,0.5110,0.28104,0.39602,0.40516,0.733521
22,4,linear,0.0001,0.000001,0.351,0.638,0.756,0.2592,0.2592,0.3756,...,0.2672,0.2672,0.3806,0.5181,0.28248,0.5156,0.28988,0.40274,0.40029,0.738462
20,4,linear,0.0001,0.000005,0.350,0.638,0.756,0.2588,0.2588,0.3760,...,0.2674,0.2674,0.3806,0.5176,0.28224,0.5161,0.29004,0.40307,0.39992,0.739429
12,2,linear,0.0001,0.000010,0.347,0.646,0.755,0.2588,0.2588,0.3794,...,0.2594,0.2594,0.3684,0.5183,0.28292,0.5037,0.28120,0.39245,0.40061,0.745821
16,2,linear,0.0001,0.000001,0.345,0.645,0.755,0.2586,0.2586,0.3788,...,0.2586,0.2586,0.3678,0.5170,0.28264,0.5045,0.28044,0.39247,0.39982,0.749565
14,2,linear,0.0001,0.000005,0.344,0.646,0.757,0.2588,0.2588,0.3792,...,0.2592,0.2592,0.3680,0.5172,0.28288,0.5039,0.28096,0.39243,0.40004,0.752326
21,4,mlp,0.0001,0.000005,0.345,0.613,0.737,0.2604,0.2604,0.3812,...,0.2424,0.2424,0.3590,0.5038,0.28456,0.4719,0.26572,0.36881,0.39418,0.754783
